In [ ]:
from exod.processing.detector import Detector, plot_var_with_regions, plot_region_lightcurves
from exod.pre_processing.data_loader import DataLoader
from exod.utils.path import save_df
from exod.xmm.observation import Observation
from exod.processing.detector import Detector
from exod.utils.logger import logger
from exod.utils.plotting import cmap_image
from photutils.detection import daofinder, irafstarfinder, starfinder

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture
from astropy.stats import sigma_clipped_stats

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu, threshold_local

In [ ]:
obsid = '0670170501'
obsid = '0803990501' # 1313 bright
obsid = '0911990501' # dim sources


size_arcsec   = 10.0
time_interval = 50
gti_only      = True
gti_threshold = 0.5
min_energy    = 0.2
max_energy    = 12.0
sigma         = 5
clobber       = False


# Create the Observation class
observation = Observation(obsid)
observation.get_files()

# Get the eventslist & image to use
event_list = observation.events_processed_pn[0]
event_list.read()

img = observation.images[0]
img.read(wcs_only=True)

# Initialize the Data Loader
dl = DataLoader(event_list=event_list,
                size_arcsec=size_arcsec,
                time_interval=time_interval,
                gti_only=gti_only,
                gti_threshold=gti_threshold,
                min_energy=min_energy,
                max_energy=max_energy)
dl.run()

# Create Data Cube
# data_cube.plot_cube_statistics()
# data_cube.video()

# Detection
detector = Detector(data_cube=dl.data_cube, wcs=img.wcs, sigma=sigma)
detector.run()
plot_var_with_regions(var_img=detector.image_var, df_regions=detector.df_regions, savepath=observation.path_results / 'image_var.png')
plot_region_lightcurves(df_lcs=detector.df_lcs, df_regions=detector.df_regions, savedir=observation.path_results)

# Save Results
save_df(df=dl.df_bti, savepath=observation.path_results / 'bti.csv')
save_df(df=detector.df_lcs, savepath=observation.path_results / 'lcs.csv')
save_df(df=detector.df_regions, savepath=observation.path_results / 'regions.csv')
obs_info = observation.info
evt_info = event_list.info
dl_info = dl.info
dc_info = dl.data_cube.info
det_info = detector.info
plt.show()





In [ ]:
#image>np.mean(image)

In [ ]:
image = detector.image_var
mean = np.nanmean(image)
std  = np.nanstd(image)
print(mean, std)

sigma=10
global_thresh= mean + sigma*std
local_thresh = threshold_local(image, block_size=3, offset=0)
image_mask   = image > local_thresh
n_regions = calc_n_regions(image_mask)

fig, ax = plt.subplots(1,3, figsize=(10,3))
ax[0].imshow(image, cmap=cmap_image(), interpolation='none')
ax[1].imshow(image > global_thresh, cmap='grey', interpolation='none')
ax[2].imshow(image > local_thresh, cmap='grey', interpolation='none')
ax[0].set_title('Image')
ax[1].set_title('Global Thresholding')
ax[2].set_title('Local Thresholding')

plt.tight_layout()
#plt.colorbar()
plt.show()

In [ ]:
image = np.where(image < global_thresh, image, np.nan)
mean = np.nanmean(image)
std  = np.nanstd(image)
print(mean, std)
#image = np.log10(image)

sigma=0.5
global_thresh= mean + sigma*std
local_thresh = threshold_local(image, block_size=3, offset=0)
image_mask   = image > local_thresh


fig, ax = plt.subplots(1,3, figsize=(10,3))
ax[0].imshow(image, cmap=cmap_image(), interpolation='none')
ax[1].imshow(image > global_thresh, cmap='grey', interpolation='none')
ax[2].imshow(image > local_thresh, cmap='grey', interpolation='none')
ax[0].set_title('Image')
ax[1].set_title('Global Thresholding')
ax[2].set_title('Local Thresholding')

plt.tight_layout()
#plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(image.flatten())

In [ ]:
import numpy as np
from skimage import img_as_ubyte
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops

def optimize_sigma(image, n_target_regions, sigma_lower, sigma_upper, tol=1e-4, max_iter=100):
    def binary_search(lower, upper, tol, max_iter):
        for i in range(max_iter):
            sigma = (lower + upper) / 2.0
            thresh = np.mean(image) + sigma * np.std(image)
            image_mask = image > thresh
            labeled_image = label(image_mask)
            regions = regionprops(labeled_image)
            n_regions = len(regions)

            print(f'i={i}, sigma={sigma}, thresh={thresh}, n_regions={n_regions}')
            
            if n_regions < n_target_regions:
                lower = sigma
            else:
                upper = sigma
            if upper - lower < tol:
                break

        return sigma
    
    optimal_sigma = binary_search(sigma_lower, sigma_upper, tol, max_iter)

    return optimal_sigma

# Example usage:
# Assuming you have an image called 'your_image'
# and you want to achieve 10 regions
sigma_lower_bound = 0.1
sigma_upper_bound = 20.0
optimal_sigma = optimize_sigma(image, n_target_regions=10, sigma_lower=sigma_lower_bound, sigma_upper=sigma_upper_bound)
print(f"Optimal Sigma: {optimal_sigma}")

In [ ]:
from scipy.optimize import minimize_scalar

In [ ]:
import numpy as np
from skimage import img_as_ubyte
from skimage.measure import label
from skimage.filters import threshold_otsu
from scipy.optimize import minimize_scalar

def calc_threshold(mean, sigma, std):
    return mean + sigma * std

def objective_function(sigma, image, image_mean, image_std, n_target_regions):
    threshold = calc_threshold(image_mean, image_std, sigma)
    image_mask = image > threshold
    labeled_image = label(image_mask)
    regions = regionprops(labeled_image)
    n_regions = len(regions)
    print(f'Optimising: sigma={sigma:.4f} threshold={threshold:.4f} n_regions={n_regions} target={n_target_regions}')
    return np.abs(n_regions - n_target_regions)

def optimize_sigma(image, image_mean, image_std, n_target_regions, sigma_bounds=(0.1, 20.0)):
    result = minimize_scalar(objective_function, args=(image, image_mean, image_std, n_target_regions), bounds=sigma_bounds, method='bounded')
    print(result)
    return result.x

sigma_opt = optimize_sigma(image, image_mean, image_std, n_target_regions=10)
threshold_opt = calc_threshold(image_mean, sigma_opt, image_std)
print(f"sigma_opt: {sigma_opt} threshold_opt: {threshold_opt}")


In [ ]:
image = detector.image_var
image_mean = np.nanmean(image)
image_std  = np.nanstd(image)

In [ ]:
plt.imshow(image > threshold_opt, cmap='grey', interpolation='none')

In [ ]:
?minimize_scalar